In [1]:
!which python

/Users/pushkalm11/miniconda3/envs/unrolling/bin/python


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from numpy.linalg import matrix_power as MP

In [3]:
import random
import scipy
from scipy.spatial.distance import cdist
from scipy.io import savemat, loadmat

In [4]:
import networkx as nx
import sys

In [5]:
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.layers import Flatten, Dense
from keras import backend as K
from keras.callbacks import EarlyStopping

In [6]:
print(f"TF Version: {tf.__version__}")

TF Version: 2.14.0


In [7]:
X_train = loadmat('./Dataset/X_train.mat')['X']
N, T = X_train.shape

In [8]:
dt = np.zeros([T - 1, T])

for i in range(T - 1):
    dt[i, i] = -1
    dt[i, i - 1] = 1

d = dt.T
d = np.concatenate((d, np.zeros([T, 1])), axis = 1) 
d[T - 1, T - 1] = -1
d = d.T
d[T - 1, T - 1] = 1 
d[-1, -2] = 1
d[0, -1] = 0
d = tf.convert_to_tensor(d, dtype = "float32")
print(d)

tf.Tensor(
[[-1.  0.  0. ...  0.  0.  0.]
 [ 1. -1.  0. ...  0.  0.  0.]
 [ 0.  1. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ...  0.  1.  1.]], shape=(152, 152), dtype=float32)


In [9]:
def adj_to_laplacian(AdjL):
    tmp = np.array(AdjL, dtype = "float32")
    return np.diag(np.sum(tmp, axis = 0)) - tmp

In [10]:
def initial_graph(X_training, N, top_values):
    # top_values = 6
    distance_matrix = cdist(X_training, X_training, metric = 'euclidean')
    sorted_indices = distance_matrix.argsort(axis = 1)
    
    TAdj = np.zeros([N, N])
    
    for i in range(N):
        for j in range(1, top_values):
            k = sorted_indices[i][j]
            TAdj[i][k] = 1
            
    TAdj = (TAdj + TAdj.T) / 2
    TAdj = np.where(TAdj > 0.1, 1, 0)
    AdjL = np.diag(np.sum(TAdj, 1)) - TAdj
    
    return AdjL

In [11]:
psi_cost = 0
psi_cost_1 = 0

In [12]:
# A - Predicted, B - Target matrix
def costfunc(A, B):
    # (y_true, y_pred)
    # print(A.shape)
    # print(B.shape)
    return (tf.norm(tf.multiply(psi_cost, (A - B)))) ** 2

# def costfunc(A, B):
#     return (tf.norm(tf.multiply(psi_cost, (A - B))) ** 2) + (reg * tf.norm(HLG @ B @ GLT) ** 2 / 2)

def mse_unknown(A, B):

    global psi_cost
    psi = psi_cost

    psid = tf.convert_to_tensor(np.ones([N, T], dtype = float) - psi, dtype = "float32")
    mse_unk = (LA.norm(np.multiply(psid, (X_train - B)))) ** 2 / np.sum(psid.numpy().flatten())
    
    global mse_unknown_list
    mse_unknown_list.append(mse_unk)
    
    return mse_unk

def mse_known(A, B):

    global psi_cost 
    psi = psi_cost

    mse_kn = (LA.norm(np.multiply(psi, (X_train - B)))) ** 2 / np.sum(psi.numpy().flatten())
    
    global mse_known_list
    mse_known_list.append(mse_kn)

    return mse_kn

In [13]:
callb = EarlyStopping(monitor = 'loss', mode = 'min', min_delta = 0.00001, verbose = 1, patience = 10)

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr / lr_decay_factor

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [14]:
test_list = []

In [15]:
class EldarModel(tf.keras.Model):
    
    def __init__(self, Laplacian, reg_init, loops_init, d_init, **kwargs):
        super(EldarModel, self).__init__(**kwargs)
        p_init = 20.0
        self.p1_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p1 = tf.Variable(initial_value = self.p1_init, trainable = True, name = "p1")
        self.p2_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p2 = tf.Variable(initial_value = self.p2_init, trainable = True, name = "p2")
        self.p3_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p3 = tf.Variable(initial_value = self.p3_init, trainable = True, name = "p3")
        self.p4_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p4 = tf.Variable(initial_value = self.p4_init, trainable = True, name = "p4")
        
        self.lap = Laplacian
        self.reg = reg_init
        self.loops = loops_init
        self.d_matrix = d_init
        
    def call(self, y):
        
        psi = y[0, :, T:]
        global psi_cost
        psi_cost = psi
        
        psi = tf.convert_to_tensor(psi, dtype = "float32")
        
        X_in = y[0, :, : T]
        X_in = tf.convert_to_tensor(X_in, dtype = "float32")
        
        Y = X_in
        
        LG, reg, loop, d = self.lap, self.reg, self.loops, self.d_matrix
        
        LT = tf.matmul(tf.transpose(d), d)
        LT = tf.convert_to_tensor(LT, dtype = "float32")
        
        HLG = tf.eye(tf.shape(LG)[0], dtype = "float32") 
        GLT = LT + (self.p1 * MP(LT, 2)) + (self.p2 * MP(LT, 3)) + (self.p3 * MP(LT, 4)) + (self.p4 * MP(LT, 5))
        
        Xk = tf.zeros_like(X_in)
        Zk = tf.zeros_like(X_in)
        Zk = -(tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT)))
        
        for i in range(loop):
            # print(tf.multiply(psi, Xk) - Y, (HLG @ Xk @ GLT))
            fdx_xk = tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT))
            fdx_zk = tf.multiply(psi, Zk) - Y + (reg * (HLG @ Zk @ GLT))
            
            tau = tf.linalg.trace(tf.transpose(fdx_xk) @ Zk) / tf.linalg.trace(tf.transpose((Y + fdx_zk)) @ Zk)
            
            Xk_1 = Xk - (tau * Zk)
            
            fdx_xk_1 = tf.multiply(psi, Xk_1) - Y + (reg * (HLG @ Xk_1 @ GLT))
            
            gamma = (tf.norm(fdx_xk_1) ** 2) / (tf.norm(fdx_xk) ** 2)
            
            Zk_1 = (gamma * Zk) - fdx_xk_1
            
            Xk = Xk_1
            Zk = Zk_1
        
        global test_list
        test_list.append(Xk_1.numpy())
        
        return tf.reshape(Xk_1, [1, N, T])

In [16]:
no_of_psi = 20
Laplacian = initial_graph(X_train, N, 12)

# sensing_ratio = [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5]
sensing_ratio = list(np.arange(0.01, 0.51, 0.01))

# reg_list = np.load("./Outputs/eldar_KNN_reg.npy")
# loops_list = np.load("./Outputs/eldar_KNN_loops.npy")

reg_list = [1.0e-5] * len(sensing_ratio)
loops_list = [50] * len(sensing_ratio)

all_psi = []

mse_known_list = []
mse_unknown_list = []

learned_variables = []

lr_decay_factor = 1.01

In [17]:
for i_sen, rem in enumerate(sensing_ratio):
    
    print(f"\n\n###########################################################################################")
    print(f"################################### Sensing Ratio: {rem * 100}% ###################################")
    print(f"###########################################################################################\n\n")
    
    if i_sen == 0:
        M = int(rem * T)
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2*T])
    
        for i in range(no_of_psi):
            psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
            np.random.shuffle(psi_k)
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
        
        psi_cost_1 = psi_k
    
    else:
        previous_psi = all_psi[-no_of_psi : ]
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])
        
        rem_diff = int(N * (rem - sensing_ratio[i_sen - 1]) * T)
        
        for i in range(no_of_psi):
            psi_k_tmp = np.array(previous_psi[i]).flatten()
            ones_idx = np.where(psi_k_tmp == 1)
            
            ones_to_zero_idx = np.random.choice(ones_idx[0], size = rem_diff)
            psi_k_tmp[ones_to_zero_idx] = 0
            psi_k = psi_k_tmp
            
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
        
    reg_init = reg_list[i_sen]
    loops_init = loops_list[i_sen]
    
    sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 1.0e-3, momentum = 0.1)
    
    eldar_model = EldarModel(Laplacian, reg_init, loops_init, d, name = 'EM')
    eldar_model.compile(optimizer = sgd_optimizer, loss = costfunc, 
                        metrics = [mse_known, mse_unknown], run_eagerly = True)
    eldar_model.fit(X_train_concatenated, X_train_missing, epochs = 100, 
                    callbacks = [lr_scheduler], batch_size = 1)
    eldar_model.summary()
    
    all_variables = eldar_model.variables
    
    graph_vars = [i.numpy()[0, 0] for i in all_variables]
    learned_variables.append(graph_vars)
    
    print(f"Learned Variables: {graph_vars}")



###########################################################################################
################################### Sensing Ratio: 1.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 8s 372ms/step - loss: 5.6971 - mse_known: 0.0012 - mse_unknown: 0.3245 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 7s 368ms/step - loss: 5.6943 - mse_known: 0.0012 - mse_unknown: 0.3245 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 7s 363ms/step - loss: 5.6914 - mse_known: 0.0012 - mse_unknown: 0.3245 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 7s 365ms/step - loss: 5.6885 - mse_known: 0.0012 - mse_unknown: 0.3245 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 7s 364ms/step - loss: 5.6857 - mse_known: 0.0012 - mse_unknown: 0.3245 - lr: 0.0010
Epoch 6/100
20/20 [================

Learned Variables: [19.98932, 19.961506, 19.858236, 19.471472]


###########################################################################################
################################### Sensing Ratio: 2.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 13s 675ms/step - loss: 5.5168 - mse_known: 0.0012 - mse_unknown: 0.3518 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 13s 674ms/step - loss: 5.5141 - mse_known: 0.0012 - mse_unknown: 0.3518 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 14s 692ms/step - loss: 5.5114 - mse_known: 0.0012 - mse_unknown: 0.3518 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 13s 635ms/step - loss: 5.5087 - mse_known: 0.0012 - mse_unknown: 0.3518 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 13s 643ms/step - loss: 5.5060 - mse_known: 0.0012 - 

Learned Variables: [19.989496, 19.96263, 19.862392, 19.48793]


###########################################################################################
################################### Sensing Ratio: 3.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 14s 678ms/step - loss: 5.3468 - mse_known: 0.0011 - mse_unknown: 0.3464 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 14s 721ms/step - loss: 5.3443 - mse_known: 0.0011 - mse_unknown: 0.3464 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 16s 779ms/step - loss: 5.3418 - mse_known: 0.0011 - mse_unknown: 0.3464 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 16s 788ms/step - loss: 5.3393 - mse_known: 0.0011 - mse_unknown: 0.3464 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 15s 764ms/step - loss: 5.3367 - mse_known: 0.0011 - m

Learned Variables: [19.989664, 19.963528, 19.866245, 19.503523]


###########################################################################################
################################### Sensing Ratio: 4.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 7s 368ms/step - loss: 5.1485 - mse_known: 0.0011 - mse_unknown: 0.3669 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 7s 371ms/step - loss: 5.1462 - mse_known: 0.0011 - mse_unknown: 0.3669 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 7s 369ms/step - loss: 5.1438 - mse_known: 0.0011 - mse_unknown: 0.3669 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 7s 368ms/step - loss: 5.1415 - mse_known: 0.0011 - mse_unknown: 0.3669 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 7s 367ms/step - loss: 5.1392 - mse_known: 0.0011 - mse_

Learned Variables: [19.98991, 19.96463, 19.870806, 19.521662]


###########################################################################################
################################### Sensing Ratio: 5.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 8s 402ms/step - loss: 4.9182 - mse_known: 0.0011 - mse_unknown: 0.4010 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 8s 396ms/step - loss: 4.9161 - mse_known: 0.0011 - mse_unknown: 0.4010 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 8s 394ms/step - loss: 4.9139 - mse_known: 0.0011 - mse_unknown: 0.4010 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 8s 401ms/step - loss: 4.9118 - mse_known: 0.0011 - mse_unknown: 0.4010 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 8s 399ms/step - loss: 4.9097 - mse_known: 0.0011 - mse_un

...

Learned Variables: [19.997252, 19.993444, 19.979832, 19.934914]


###########################################################################################
################################### Sensing Ratio: 46.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 9s 429ms/step - loss: 0.6579 - mse_known: 2.4498e-04 - mse_unknown: 0.6307 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 8s 398ms/step - loss: 0.6578 - mse_known: 2.4497e-04 - mse_unknown: 0.6307 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 8s 400ms/step - loss: 0.6578 - mse_known: 2.4495e-04 - mse_unknown: 0.6307 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 8s 400ms/step - loss: 0.6577 - mse_known: 2.4494e-04 - mse_unknown: 0.6307 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 8s 399ms/step - loss: 0.6577 - mse_kno

Learned Variables: [19.997438, 19.993862, 19.981058, 19.93925]


###########################################################################################
################################### Sensing Ratio: 47.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 8s 405ms/step - loss: 0.6237 - mse_known: 2.3646e-04 - mse_unknown: 0.6340 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 8s 406ms/step - loss: 0.6237 - mse_known: 2.3645e-04 - mse_unknown: 0.6340 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 8s 407ms/step - loss: 0.6237 - mse_known: 2.3644e-04 - mse_unknown: 0.6340 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 8s 403ms/step - loss: 0.6236 - mse_known: 2.3642e-04 - mse_unknown: 0.6340 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 8s 407ms/step - loss: 0.6236 - mse_know

Learned Variables: [19.99758, 19.99409, 19.98201, 19.942446]


###########################################################################################
################################### Sensing Ratio: 48.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 8s 406ms/step - loss: 0.5860 - mse_known: 2.2626e-04 - mse_unknown: 0.6367 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 9s 427ms/step - loss: 0.5860 - mse_known: 2.2625e-04 - mse_unknown: 0.6367 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 8s 413ms/step - loss: 0.5860 - mse_known: 2.2623e-04 - mse_unknown: 0.6367 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 8s 404ms/step - loss: 0.5859 - mse_known: 2.2622e-04 - mse_unknown: 0.6367 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 8s 402ms/step - loss: 0.5859 - mse_known:

Learned Variables: [19.997782, 19.994398, 19.983099, 19.94601]


###########################################################################################
################################### Sensing Ratio: 49.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 8s 393ms/step - loss: 0.5426 - mse_known: 2.1339e-04 - mse_unknown: 0.6408 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 10s 476ms/step - loss: 0.5426 - mse_known: 2.1338e-04 - mse_unknown: 0.6408 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 8s 400ms/step - loss: 0.5425 - mse_known: 2.1337e-04 - mse_unknown: 0.6408 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 8s 384ms/step - loss: 0.5425 - mse_known: 2.1336e-04 - mse_unknown: 0.6408 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 8s 395ms/step - loss: 0.5425 - mse_kno

Learned Variables: [19.99798, 19.994661, 19.984274, 19.950045]


###########################################################################################
################################### Sensing Ratio: 50.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 8s 406ms/step - loss: 0.5044 - mse_known: 2.0215e-04 - mse_unknown: 0.6382 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 8s 400ms/step - loss: 0.5044 - mse_known: 2.0214e-04 - mse_unknown: 0.6382 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 7s 369ms/step - loss: 0.5044 - mse_known: 2.0213e-04 - mse_unknown: 0.6382 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 9s 435ms/step - loss: 0.5043 - mse_known: 2.0212e-04 - mse_unknown: 0.6382 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 8s 377ms/step - loss: 0.5043 - mse_know

In [18]:
for i, rem in enumerate(sensing_ratio):
    print(f"For {int(100 * rem)}% sensing ratio: {learned_variables[i]}")

For 1% sensing ratio: [19.98932, 19.961506, 19.858236, 19.471472]
For 2% sensing ratio: [19.989496, 19.96263, 19.862392, 19.48793]
For 3% sensing ratio: [19.989664, 19.963528, 19.866245, 19.503523]
For 4% sensing ratio: [19.98991, 19.96463, 19.870806, 19.521662]
For 5% sensing ratio: [19.990318, 19.96607, 19.876152, 19.542883]
For 6% sensing ratio: [19.990618, 19.966965, 19.880056, 19.558086]
For 7% sensing ratio: [19.99096, 19.967884, 19.88343, 19.571379]
For 8% sensing ratio: [19.991295, 19.968845, 19.887291, 19.586552]
For 9% sensing ratio: [19.991484, 19.969648, 19.890648, 19.599785]
For 10% sensing ratio: [19.99167, 19.970432, 19.893934, 19.612614]
For 11% sensing ratio: [19.991842, 19.971342, 19.897228, 19.625698]
For 12% sensing ratio: [19.992027, 19.97217, 19.90072, 19.639307]
For 13% sensing ratio: [19.992266, 19.973175, 19.904552, 19.654125]
For 14% sensing ratio: [19.99248, 19.974052, 19.907963, 19.66733]
For 15% sensing ratio: [19.992704, 19.974953, 19.911526, 19.681076]
Fo

In [19]:
np.save('./Outputs/eldar_KNN_mse_known.npy', mse_known_list)
np.save('./Outputs/eldar_KNN_mse_unknown.npy', mse_unknown_list)